# Demo: OpenAI Large Language Model Chain of Thoughts Demo

이 데모에서는 GPT3 모델을 사용하여 자연 쿼리를 분석하고 지식 기반을 사용하여 더 많은 정보를 검색하고 질문에 답변하는 방법을 보여줍니다.  
1. Bing Search API 활용: 검색한 결과를 참고하는 방법을 소개합니다. 여기서는 Bing Search API를 활용합니다.
2. Azure AI Search 활용: 첫 날 만든 Wikipedia 검색용 엔진 정보를 활용하는 방법을 살펴봅니다.

In [45]:
import os
import json
import requests
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name = os.getenv('DEPLOYMENT_NAME')

## 1. Bing Search API를 활용하는 방법

### Start with a natural question  
다음과 같이 LLM이 가지고 있지 않을 수 있는 데이터에 대한 접근이 필요할 때, 검색엔진을 활용할 수 있습니다.

In [46]:
input = "2024년 한국 총선의 날짜는 언제인지 회사를 안 가도 되는지 알려줘."

### Step 1: GPT3: What do I need to to answer this question?

In [47]:
system_prompt='''Answer the following questions as best you can. You have access to the following tools: You can only Korean language.

Web Search: Use the web to find information

Question: the input question you must answer
The format of your response must strictly follow the JSON format below.
{"Thought":"You should always think about what to do.","Action":"The action to be taken must be one of the following: [Research]","KeyforSearch": "Keywords for search"}
'''
user_prompt = f'''
Question: {input}
'''

In [48]:
response = client.chat.completions.create(
    model=deployment_name,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": "Question: Predict which country will win the most medals at the 2024 Olympic Games based on the results of the past three Olympics. Explain the reasons.",},
                {"role":"assistant","content": """{"Thought":"To predict which country will win the most medals at the 2024 Olympic Games based on past results, I should research the medal counts of countries in the past three Olympics to identify trends and dominant countries. I can also consider factors like population size, investment in sports, and performance in recent international competitions.","Action":"Research","KeyforSearch": '"2024 Olympic Games medal predictions" + "country with most medals in last three Olympics" + "Olympic medal count trends"'}"""},
                {"role":"user","content": user_prompt,}],
        max_tokens=600,
        response_format={"type": "json_object"}
)

print(response.choices[0].message.content)

{"Thought":"I should search for the scheduled date of the 2024 general election in South Korea to provide the accurate information to the user.","Action":"Research","KeyforSearch": "2024 한국 총선 일정"}


In [49]:
thought = json.loads(response.choices[0].message.content)['Thought']
Action = json.loads(response.choices[0].message.content)['Action']
KeyforSearch = json.loads(response.choices[0].message.content)['KeyforSearch']

print(thought)
print(KeyforSearch)

I should search for the scheduled date of the 2024 general election in South Korea to provide the accurate information to the user.
2024 한국 총선 일정


### Step 2: Search web for more details  

Bing Search API와 관련된 정보는 다음 Page에서 확인할 수 있습니다.  
https://www.microsoft.com/en-us/bing/apis/bing-web-search-api

In [50]:
# Use Azure BING Search API to get the results
subscription_key = os.environ['BING_SEARCH_KEY']
endpoint = os.environ['BING_SEARCH_ENDPOINT'] + "/v7.0/search"

query = KeyforSearch

# Construct a request
mkt = 'ko_KR'
params = { 'q': query, 'mkt': mkt }
headers = { 'Ocp-Apim-Subscription-Key': subscription_key }

# Call the API
response = requests.get(endpoint, headers=headers, params=params)

context = ''
for result in response.json()['webPages']['value']:
    context += result['snippet'] + '\n'

# params = {"q": "2024 presidential election candidates and current political analysis"}
# response = requests.get(url, headers=headers, params=params)


In [51]:
context = ''
for result in response.json()['webPages']['value']:
    context += result['snippet'] + '\n'

In [52]:
print(context)

목차 보이기. 2024년 총선의 일정. 총선은 현 정부의 국정운영을 평가하고 국민의 의견을 반영하는 중요한 중간평가이므로 결과에 대해 많은 주목을 받고 있습니다. 22대 국회의원 선거 날짜는 다음과 같습니다. 사전투표: 4월 5일 (목)부터 4월 6일 (금)까지. 본투표: 4월 10일 (수) 투표 시간은 사전투표와 본투표 모두 오전 6시부터 오후 6시까지이며, 총선일은 법정 공휴일로 지정되어 있습니다. 아래에서는 2024년 총선의 사전투표 일정과 투표 방법에 대해 자세히 살펴보겠습니다. 22대 국회의원 선거 사전투표 일정. 사전투표 기간: 2024년 4월 5일부터 4월 6일까지.
2024년 3월 21일 ~ 2024년 3월 22일 까지. 후보자 등록 및 신청 (선거일전 20일부터 2일간) 2024년 3월 27일 까지. 선거벽보 제출 (후보자등록 마감일 후 5일까지) 2024년 3월 28일. 선거기간 개시일 (후보자등록 마감일 후 6일) 2024년 3월 29일
선거사무안내. 내 선거구 검색. [선거정보시스템 점검 알림] 2024. 3. 23. (토) 21:00 ~ 24. (일) 09:00. 3.21. ~ 3. 22. 제22대 국회의원선거 후보자등록. 거소투표·선상투표 대상자는 3월 19일부터 23일까지 신고하세요. 중앙선관위, 제22대 국선 선거범죄 포상금 첫 지급. 중앙선관위, 선거운동 문자메시지 관련 위법행위 예방단속 강화. 다음 이전 멈춤. 이전버튼 다음버튼. 멈춤 전체보기. 선거·법령 정보. (역대) 투표율 정보 (역대)선거통계 선거일정 선거자료 정치관계법 개정법률 서면/인터넷 질의보기 판례보기 자주묻는 질문. Banner & Event. 멈춤. 정당/후보자.
2024 총선 | 제 22대 국회의원 선거와 관련된 정보를 빠르고 정확하게 알려드립니다. 여론조사, 선거뉴스, 선거영상 등 다양한 콘텐츠를 KBS 총선 특집사이트에서 확인하세요.
대한민국 제22대 국회의원 선거는 2024년 4월 10 일에 실시됩니다. 이 날은 임시 공휴일 로 지정됩니다. 후보자 사전

In [53]:
system_prompt = """You are an agent who answers the user's questions based on the data provided. You can only Korean language."""
user_prompt = f"""Question: {input}
Plan: {thought}
Web Search Results: {context}
Answer: 
"""

response = client.chat.completions.create(
    model=deployment_name,
    messages = [
        {"role":"system", "content":system_prompt},
        {"role":"user","content": user_prompt}
    ],
    max_tokens=600
)

print(response.choices[0].message.content)

2024년 4월 10일에 22대 국회의원 선거가 예정되어 있습니다. 이 날은 법정 공휴일로 설정되어 있어서 회사를 쉬는 날로 기억하시면 됩니다.


## 2. AI Search를 이용하여 나의 데이터 기반으로 검색하는 방법  

RAG 모델은 정보 검색 시스템과 언어 모델을 결합하여, 주어진 질문에 대한 답변을 생성하기 위해 관련 정보를 검색하고, 그 정보를 바탕으로 답변을 생성합니다. 이 과정에서 Azure AI Search 검색 서비스가 핵심적인 역할을 합니다.

1. 정보 검색의 효율성: Azure AI Search는 대규모 데이터 세트에서 빠르게 관련 정보를 검색할 수 있는 기능을 제공합니다. RAG 모델은 이러한 검색 엔진을 사용하여 주어진 질문과 관련된 문서나 데이터를 효율적으로 찾아내며, 이를 바탕으로 더 정확한 답변을 생성할 수 있습니다.  

2. 정보의 정확성 및 신뢰성: Azure AI Search는 다양한 최적화 기법과 알고리즘을 사용하여 검색 결과의 정확성을 높입니다. 이는 RAG 모델이 더 신뢰할 수 있는 정보를 기반으로 답변을 생성할 수 있도록 도와줍니다.  

3. 확장성 및 가용성: Azure AI Search는 클라우드 기반 서비스로, 확장성이 뛰어나고 언제 어디서나 접근할 수 있는 장점이 있습니다. RAG 모델은 이러한 클라우드 서비스를 활용하여 다양한 규모와 범위의 데이터에 대한 검색 요구를 충족시킬 수 있습니다.  

4. 사용자 정의 및 적응성: Azure AI Search는 사용자가 직접 검색 알고리즘을 세밀하게 조정하고 최적화할 수 있도록 지원합니다. 이를 통해 RAG 모델은 특정 도메인이나 질문 유형에 맞게 검색 기능을 맞춤 설정할 수 있으며, 이는 전반적인 답변의 품질을 향상시키는 데 기여합니다.  

> 아래는 다음과 같은 사전 작업을 했다는 전제하에 진행할 수 있습니다.
- 해당 검색엔진은 OpenAI에서 제공하는 백터 정보가 포함된 Wikipeia 문서의 http://azure.studydev.com/openai/vector_sample.csv 파일을 업로드하고 색인화 했습니다.
- 백터 데이터를 Azure AI Search 엔진으로 색인화하는 방법 소개 영상: https://www.youtube.com/watch?v=XsqPjHYZD04

In [54]:
# Azure Cognitive Search
# Create an SDK client
# endpoint = os.getenv("OPENAI_API_ENDPOINT")
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

endpoint    = os.getenv('AZURE_AI_SEARCH_ENDPOINT')
credential  = AzureKeyCredential(os.getenv('AZURE_AI_SEARCH_API_KEY'))
index_name  = os.getenv('AZURE_AI_SEARCH_INDEX')
api_version = os.getenv('AZURE_AI_SEARCH_API_VERSION')
embedding_model_name = "text-embedding-ada-002"

search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential, api_version=api_version)

question     = "4월과 8월은 어떤 차이점이 있는지 알려주세요."
embedding    = client.embeddings.create(input = question, model=embedding_model_name).data[0].embedding
vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3, fields="content_vector", exhaustive=True)

results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["id", "title", "text"],
)  


### Collected information for further processing

In [55]:
grounding = ""
for result in results:  
    # print(f"Title: {result['title']}")  
    # print(f"Score: {result['@search.score']}")  
    # print(f"Text:  {result['text']}")  
    # print(f"id:    {result['id']}\n")  
    grounding = grounding + " \n " + result['text']

In [56]:
input = question
prompt=f'''Answer the following questions as best you can. You have access to the following tools:
Question: {input}
Gronding data: {grounding}
Thought:'''


In [57]:
print(prompt)

Answer the following questions as best you can. You have access to the following tools:
Question: 4월과 8월은 어떤 차이점이 있는지 알려주세요.
Gronding data:  
 August (Aug.) is the eighth month of the year in the Gregorian calendar, coming between July and September. It has 31 days. It is named after the Roman emperor Augustus Caesar.
August does not begin on the same day of the week as any other month in common years, but begins on the same day of the week as February in leap years. August always ends on the same day of the week as November.
The Month 
This month was first called Sextilis in Latin, because it was the sixth month in the old Roman calendar. The Roman calendar began in March about 735 BC with Romulus. October was the eighth month. August was the eighth month when January or February were added to the start of the year by King Numa Pompilius about 700 BC. Or, when those two months were moved from the end to the beginning of the year by the decemvirs about 450 BC (Roman writers disagree). 

### Step 3: GPT3: What do I need to do to answer this question?

In [60]:
response = client.chat.completions.create(
    model=deployment_name,
    messages = [
        {"role":"system", "content":"You are a helpful  assistant. If you have data that can be expressed in a table, use Markdown to print it as a table. You can only Korean language. "},
        {"role":"user","content": prompt}
    ],
    max_tokens=600
)

print(response.choices[0].message.content)

| 항목       | 4월                                               | 8월                                                   |
|-----------|---------------------------------------------------|-------------------------------------------------------|
| 월의 순서  | 4번째                                            | 8번째                                                 |
| 일수       | 30일                                               | 31일                                                   |
| 시작 요일  | 7월과 같음 (평년), 윤년인 경우 1월과 같음          | 7월과 다름 (평년), 윤년인 경우 2월과 같음              |
| 종료 요일  | 12월과 같음                                         | 11월과 같음                                           |
| 꽃         | 무당벌레와 데이지                                     | 글라디올러스                                             |
| 탄생석     | 다이아몬드                                          | 페리도트                                                |
| 문자의 의미 | 결백                                                | 힘                          